In [1]:
library(tidyverse)
library(ggplot2)
library(PropCIs)
library(zeallot)
library(DBI)
library(vcd)
con <- DBI::dbConnect(odbc::odbc(), driver = "/usr/local/Cellar/psqlodbc/13.02.0000/lib/psqlodbcw.so", database = "yukontaf", UID = "glebsokolov", host = "localhost",
  port = 5432)

── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Загрузка требуемого пакета: grid



In [2]:
credit_score <- dbSendQuery(con, "SELECT * FROM credit_score")
credit_score <- dbFetch(credit_score)

Let's convert the columns in the receieved data to the right types.

In [3]:
for (col in 1:ncol(credit_score)) {
  colnames(credit_score)[col] <- tolower(colnames(credit_score)[col])
}

In [5]:
credit_score <- subset(credit_score, select = -c(index, id))
categories <- c("sex", "education", "marriage", "default")
for (col in categories) {
  credit_score[, col] <- as.factor(credit_score[, col])
}
numerical <- names(subset(credit_score, select = -c(sex, education, marriage, default)))
for (n in numerical) {
  credit_score[, n] <- as.double(credit_score[, n])
}

In [9]:
credit_score

index,id,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,⋯,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default
<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,⋯,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>
0,1,20000,2,2,1,24,2,2,0,⋯,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,0,2,0,⋯,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,⋯,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,⋯,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,0,0,0,⋯,20940,19146,19131,2000,36681,10000,9000,689,679,0
5,6,50000,1,1,2,37,0,0,0,⋯,19394,19619,20024,2500,1815,657,1000,1000,800,0
6,7,500000,1,1,2,29,0,0,0,⋯,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
7,8,100000,2,2,2,23,0,0,0,⋯,221,-159,567,380,601,0,581,1687,1542,0
8,9,140000,2,3,1,28,0,0,2,⋯,12211,11793,3719,3329,0,432,1000,1000,1000,0


Let's test two hypothesis: 
- Are the mean credit limits (limit_bal) value for two groups default = 0 (didn't returned the credit) and default = 1 equal to each other?
- Are the distributions of the limit_bal for these two groups also equal to each other?

In [6]:
p <- ggplot(credit_score, aes(x = default, y = limit_bal)) + geom_boxplot()
t.test(limit_bal ~ default, data = credit_score)
wilcox.test(limit_bal ~ default, data = credit_score)


	Welch Two Sample t-test

data:  limit_bal by default
t = 28.952, df = 11982, p-value < 2.2e-16
alternative hypothesis: true difference in means between group 0 and group 1 is not equal to 0
95 percent confidence interval:
 44740.91 51239.23
sample estimates:
mean in group 0 mean in group 1 
       178099.7        130109.7 



	Wilcoxon rank sum test with continuity correction

data:  limit_bal by default
W = 95786286, p-value < 2.2e-16
alternative hypothesis: true location shift is not equal to 0


**These results are obviously practically significant.**

Now, lets test another pair of hypothesis:
- Are the mean ages and their distributions for these two groups equal to each other?

In [7]:
t.test(age ~ default, credit_score)
wilcox.test(age ~ default, credit_score)


	Welch Two Sample t-test

data:  age by default
t = -2.3195, df = 10173, p-value = 0.02039
alternative hypothesis: true difference in means between group 0 and group 1 is not equal to 0
95 percent confidence interval:
 -0.56915863 -0.04778641
sample estimates:
mean in group 0 mean in group 1 
       35.41727        35.72574 



	Wilcoxon rank sum test with continuity correction

data:  age by default
W = 76966880, p-value = 0.3725
alternative hypothesis: true location shift is not equal to 0


The result we received tells us that statistically, mean ages *are* different, but from the confidence interval value we can see that **this difference is hardly practically signigicant.**

Now let's see if the gender composition for the two groups differ.

In [6]:
good <- filter(credit_score, default == 0)
bad <- filter(credit_score, default == 1)
c(ngoodmen, total_good, nbadmen, total_bad) %<-% c(table(good$sex)[1], sum(table(good$sex)), table(bad$sex)[1], sum(table(bad$sex)))
diffscoreci(ngoodmen, total_good, nbadmen, total_bad, conf.level = 0.95)




data:  

95 percent confidence interval:
 -0.06057240 -0.03366348


That means that men do not return their credits **slightly more often** (3-6%) than women.

Now, let's see if the education level impacts default rate.

In [13]:
crosstab <- table(credit_score$education, credit_score$default)
crosstab
crosstab - chisq.test(crosstab)$expected
chisq.test(crosstab)
assocstats(crosstab)

   
        0     1
  0    14     0
  1  8549  2036
  2 10700  3330
  3  3680  1237
  4   116     7
  5   262    18
  6    43     8

Warning message in chisq.test(crosstab):
“Chi-squared approximation may be incorrect”


   
            0         1
  0    3.0968   -3.0968
  1  305.4020 -305.4020
  2 -226.5640  226.5640
  3 -149.3596  149.3596
  4   20.2076  -20.2076
  5   43.9360  -43.9360
  6    3.2812   -3.2812

Warning message in chisq.test(crosstab):
“Chi-squared approximation may be incorrect”



	Pearson's Chi-squared test

data:  crosstab
X-squared = 163.22, df = 6, p-value < 2.2e-16


                    X^2 df P(> X^2)
Likelihood Ratio 184.71  6        0
Pearson          163.22  6        0

Phi-Coefficient   : NA 
Contingency Coeff.: 0.074 
Cramer's V        : 0.074 

Finally, let's see if the marriage category impacts the default category.

In [14]:
marriage_crosstab <- table(credit_score$marriage, credit_score$default)
marriage_crosstab
assocstats(marriage_crosstab)

   
        0     1
  0    49     5
  1 10453  3206
  2 12623  3341
  3   239    84

                    X^2 df   P(> X^2)
Likelihood Ratio 36.609  3 5.5663e-08
Pearson          35.662  3 8.8259e-08

Phi-Coefficient   : NA 
Contingency Coeff.: 0.034 
Cramer's V        : 0.034 

For both variables (education and marriage) we see that **they statitically significant impact the default category**. However, the contigency coefficients (which tells us how strong the features are correlated) are relatively small.